In [1]:
# Import the NumPy module.
import pandas as pd
import numpy as np
import requests
import time

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# List for holding lat_lngs and cities

lat_lngs = []
cities =[]

# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=20)
lngs = np.random.uniform(low=-180.000, high=180.000, size=20)


lat_lngs = zip(lats, lngs)

coordinates = list(lat_lngs)


# Identify nearest city for each lat, lng combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1])
    
    city_name = city.city_name
    
     # Replace spaces with %20 to create url correctly 
    city_name = city_name.replace(" ", "%20")
    
     
    # If the city is unique, then add it to a our cities list
    if city_name not in cities:

        cities.append(city_name)

# Print the city count to confirm sufficient count
len(cities)


134

In [3]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [4]:
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [11]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1
    
    #print(city_url)
    
    #city_weather = requests.get(city_url).json()
    
    #print(city_weather["weather"][0]["description"])
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        city_description = city_weather["weather"][0]["description"]
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | glamsbjerg
Processing Record 2 of Set 1 | mayo
Processing Record 3 of Set 1 | san%20patricio
Processing Record 4 of Set 1 | vanavara
Processing Record 5 of Set 1 | bluff
Processing Record 6 of Set 1 | kodiak
Processing Record 7 of Set 1 | barrow
Processing Record 8 of Set 1 | faanui
Processing Record 9 of Set 1 | talnakh
Processing Record 10 of Set 1 | rikitea
Processing Record 11 of Set 1 | puerto%20ayora
Processing Record 12 of Set 1 | badhni%20kalan
Processing Record 13 of Set 1 | vardo
Processing Record 14 of Set 1 | labuhan
Processing Record 15 of Set 1 | punta%20arenas
Processing Record 16 of Set 1 | ushuaia
Processing Record 17 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 18 of Set 1 | gonbad-e%20qabus
Processing Record 19 of Set 1 | chokurdakh
Processing Record 20 of Set 1 | busselton
Processing Record 21 of Set 1 | saskylakh
Processing Record 22 of Set 1 | ma

In [12]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Glamsbjerg,DK,55.2724,10.1048,65.64,90,55,10.07,broken clouds
1,Mayo,US,38.8876,-76.5119,80.94,80,75,3.44,broken clouds
2,San%20Patricio,US,28.0170,-97.5169,86.54,71,100,11.95,overcast clouds
3,Vanavara,RU,60.3400,102.2797,47.68,87,100,2.98,overcast clouds
4,Bluff,NZ,-46.6000,168.3333,47.21,66,50,21.68,scattered clouds
5,Kodiak,US,57.7900,-152.4072,54.91,94,100,11.50,light rain
6,Barrow,US,71.2906,-156.7887,41.02,93,100,11.50,mist
7,Faanui,PF,-16.4833,-151.7500,76.98,78,11,13.58,few clouds
8,Talnakh,RU,69.4865,88.3972,58.21,94,100,7.36,light rain
9,Rikitea,PF,-23.1203,-134.9692,70.70,72,71,17.29,broken clouds


In [16]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")